In [1]:
from geopy.distance import geodesic
import pandas as pd
import numpy as np

In [8]:
# Load the simulated locations
locations = pd.read_csv("carvana_delivery_locations.csv")
num_locations = len(locations)

In [9]:
# Define the number of vehicles in your fleet
num_vehicles = 3  # Adjust this number based on your specific scenario

# Define the index of the depot
depot_index = 0  # Assuming the depot is the first location in your list


In [10]:
distance_matrix = np.zeros((num_locations, num_locations))

for i in range(num_locations):
    for j in range(num_locations):
        coord1 = (locations.iloc[i]['lat'], locations.iloc[i]['lng'])
        coord2 = (locations.iloc[j]['lat'], locations.iloc[j]['lng'])
        distance_matrix[i][j] = geodesic(coord1, coord2).km


distance_matrix_m = (distance_matrix * 1000).astype(int)

In [11]:
def print_solution(manager, routing, solution):
    """Prints the routes for each vehicle."""
    total_distance = 0
    total_load = 0
    for vehicle_id in range(num_vehicles):
        index = routing.Start(vehicle_id)
        plan_output = f'Route for vehicle {vehicle_id}:\n'
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += demands[node_index]
            plan_output += f' {node_index} Load({route_load}) -> '
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
        plan_output += f'{manager.IndexToNode(index)} Load({route_load})\n'
        plan_output += f'Distance of the route: {route_distance}m\n'
        plan_output += f'Load of the route: {route_load}\n'
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print(f'Total distance of all routes: {total_distance}m')
    print(f'Total load of all routes: {total_load}')


In [12]:
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

# Create the routing index manager
manager = pywrapcp.RoutingIndexManager(len(distance_matrix), num_vehicles, depot_index)

# Create the routing model
routing = pywrapcp.RoutingModel(manager)

# Define the distance callback
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return distance_matrix[from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)

# Set the cost of travel for each arc
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Define vehicle capacities and demands
vehicle_capacities = [9] * num_vehicles
demands = [0] + [1] * (len(locations) - 1)

# Define the demand callback
def demand_callback(from_index):
    """Returns the demand of the node."""
    from_node = manager.IndexToNode(from_index)
    return demands[from_node]

demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)

# Add the capacity dimension
routing.AddDimensionWithVehicleCapacity(
    demand_callback_index,
    0,  # Null capacity slack
    vehicle_capacities,  # Vehicle maximum capacities
    True,  # Start cumul to zero
    'Capacity'
)

# Define search parameters
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

# Solve the problem
solution = routing.SolveWithParameters(search_parameters)

# Extract and display the solution
if solution:
    print_solution(manager, routing, solution)
else:
    print('No solution found!')


Route for vehicle 0:
 0 Load(0) -> 0 Load(0)
Distance of the route: 0m
Load of the route: 0

Route for vehicle 1:
 0 Load(0) ->  6 Load(1) ->  5 Load(2) ->  4 Load(3) ->  3 Load(4) ->  2 Load(5) ->  1 Load(6) -> 0 Load(6)
Distance of the route: 0m
Load of the route: 6

Route for vehicle 2:
 0 Load(0) ->  15 Load(1) ->  14 Load(2) ->  13 Load(3) ->  12 Load(4) ->  11 Load(5) ->  10 Load(6) ->  9 Load(7) ->  8 Load(8) ->  7 Load(9) -> 0 Load(9)
Distance of the route: 0m
Load of the route: 9

Total distance of all routes: 0m
Total load of all routes: 15


In [13]:
def get_routes(solution, routing, manager):
    routes = []
    for vehicle_id in range(num_vehicles):
        index = routing.Start(vehicle_id)
        route = []
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route.append(node_index)
            index = solution.Value(routing.NextVar(index))
        route.append(manager.IndexToNode(index))  # end node
        routes.append(route)
    return routes

if solution:
    vehicle_routes = get_routes(solution, routing, manager)
    for i, route in enumerate(vehicle_routes):
        print(f"Truck {i+1} route:")
        for idx in route:
            print(f"  -> {locations.iloc[idx]['location_id']}")


Truck 1 route:
  -> Depot
  -> Depot
Truck 2 route:
  -> Depot
  -> Delivery_6
  -> Delivery_5
  -> Delivery_4
  -> Delivery_3
  -> Delivery_2
  -> Delivery_1
  -> Depot
Truck 3 route:
  -> Depot
  -> Delivery_15
  -> Delivery_14
  -> Delivery_13
  -> Delivery_12
  -> Delivery_11
  -> Delivery_10
  -> Delivery_9
  -> Delivery_8
  -> Delivery_7
  -> Depot


In [14]:
import folium
import random

# Create a base map centered at the depot
depot_lat = locations.iloc[0]['lat']
depot_lng = locations.iloc[0]['lng']
m = folium.Map(location=[depot_lat, depot_lng], zoom_start=12)

# Define colors for routes
colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred']

folium.Marker(
    location=[depot_lat, depot_lng],
    popup='Depot: Carvana HQ',
    icon=folium.Icon(color='black', icon='home', prefix='fa')
).add_to(m)
# Plot routes
for i, route in enumerate(vehicle_routes):
    color = colors[i % len(colors)]
    route_coords = []
    
    for idx in route:
        lat = locations.iloc[idx]['lat']
        lng = locations.iloc[idx]['lng']
        route_coords.append((lat, lng))

        # Add marker for each stop
        popup = locations.iloc[idx]['location_id']
        folium.CircleMarker(
            location=(lat, lng),
            radius=6,
            color=color,
            fill=True,
            fill_opacity=0.8,
            popup=popup
        ).add_to(m)

    # Draw lines connecting the route
    folium.PolyLine(route_coords, color=color, weight=3, opacity=0.8).add_to(m)

# Save the map to an HTML file
m.save("carvana_routes_map.html")


In [18]:
vehicle_routes = get_routes(solution, routing, manager)
for i, route in enumerate(vehicle_routes):
    print(f"Truck {i+1} route: {route}")


Truck 1 route: [0, 0]
Truck 2 route: [0, 0]
Truck 3 route: [0, 4, 10, 3, 13, 2, 12, 8, 9, 1, 15, 7, 5, 11, 6, 14, 0]
